In [1]:
import pickle
import re
import numpy as np

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

age = []
fc = []
sex = []
race = []
res1 = []
res5 = []
ab1 = []
ab5 = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

for sub in demo['age_at_cnb']:
    for task in ['rest', 'nback', 'emoid']:
        if sub not in demo['Race'] or demo['Race'][sub] not in ['AA', 'EA']:
            continue
        a = demo['age_at_cnb'][sub]
        s = demo['Sex'][sub] == 'M'
        r = demo['Race'][sub] == 'AA'
        try:
            p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
        except:
            continue
        r1 = np.load(f'{demodir}/fc/{sub}_task-{task}res1_fc.npy')
        res1.append(r1)
        r5 = np.load(f'{demodir}/fc/{sub}_task-{task}res5_fc.npy')
        res5.append(r5)
        a1 = np.load(f'{demodir}/fc/{sub}_task-{task}ab1_fc.npy')
        ab1.append(a1)
        a5 = np.load(f'{demodir}/fc/{sub}_task-{task}ab5_fc.npy')
        ab5.append(a5)
        age.append(a)
        sex.append(s)
        race.append(r)
        fc.append(p)
    
race = np.array(race)
sex = np.array(sex)
age = np.array(age)
fc = np.stack(fc)
res1 = np.stack(res1)
res5 = np.stack(res5)
ab1 = np.stack(ab1)
ab5 = np.stack(ab5)

print([a.shape for a in [sex, race, age, fc, res1, res5, ab1, ab5]])
print(np.mean(age))
print(np.mean(sex))
print(np.mean(race))

[(3849,), (3849,), (3849,), (3849, 34716), (3849, 34716), (3849, 34716), (3849, 34716), (3849, 34716)]
14.398285268901013
0.473629514159522
0.4811639386853728


In [2]:
from sklearn.decomposition import PCA, FactorAnalysis

# pca = PCA(n_components=1).fit(fc)
# fc_comp = pca.transform(fc)
# fc_low = pca.inverse_transform(fc_comp)

fa = FactorAnalysis(n_components=10).fit(fc)
fa_low = fa.transform(fc)@fa.components_
fa_res = fc-fa_low

print(fa_res.shape)

(3849, 34716)


In [11]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

# aa = np.where(race == 1)[0]
# m = np.where(sex == 1)[0]
# aam = np.intersect1d(aa, m)
# idcs = np.random.permutation(aam.shape[0])[:807]
# aam = aam[idcs]
aam = np.random.permutation(fc.shape[0])[:2428]#[:807]
print(aam.shape)

accs = []

for i in range(20):
    x = fa_low[aam]
    xx = fa_res[aam]
    y = age[aam]
    
    xtr, xt, xxtr, xxt, ytr, yt = train_test_split(x, xx, y, train_size=0.8)
    
    mu = np.mean(ytr)
    ytr = ytr-mu
    yt = yt-mu
    
    reg = Ridge(alpha=1).fit(xtr, ytr)
    yhat0 = reg.predict(xt)
    reg = Ridge(alpha=1).fit(xxtr, ytr)
    yhat1 = reg.predict(xxt)
    acc = rmse((yhat0+yhat1)/2, yt)
    accs.append(acc)
    print(rmse(yhat0, yt), rmse(yhat1, yt), acc)
    
print('---')
print(np.mean(accs))
print(np.std(accs))


(2000,)
3.2186482446249762 4.159972429888769 2.9293245664224505
3.109571632835377 4.226667631472063 2.9079987175180966
3.0277865183618067 3.9978051155964347 2.8244481435638753
3.0421520092652585 4.524573551229347 3.0770082090402817
3.098697684899915 4.015779273804223 2.916702144249215
3.2017676578522565 4.490257713053606 3.0271660658684802
2.9588829716136607 3.939519674412528 2.675325037429228
3.1423167497212767 3.803690247184307 2.7363174127531322
3.2051990304693985 4.011508488483457 2.843993850602403


KeyboardInterrupt: 

In [38]:
print(np.std(accs))

0.10154195352626498


In [65]:
print(rmse(0, yt))

3.486050913704901
